### **<h2>Ресурсы и URI в REST API</h2>**

**<span style="color: #85004B;">Ресурсы</span>** — это объекты или сущности приложения, такие как пользователи, заказы и товары. Каждый ресурс представлен URI (Uniform Resource Identifier) и должен иметь четкую структуру и стандарты доступа. Разработка REST API начинается с определения структуры URI и маршрутизации для взаимодействия с ресурсами.

#### **Моделирование ресурсов и принципы URI-структурирования**

Для каждой сущности создается свой URI, с помощью которого API может выполнять операции над ресурсом, такие как получение, создание, обновление и удаление.

##### **Принципы структурирования URI**

1. **Определение ресурса и URI**

   - Каждый ресурс должен быть представлен URI во множественном числе для согласованности и предсказуемости.
   
   - Структура URI:
     - `/api/users/` — доступ к списку пользователей.
     - `/api/users/123/` — доступ к конкретному пользователю с ID 123.
     - `/api/orders/` — доступ к списку заказов.
     - `/api/orders/456/` — доступ к конкретному заказу.
   
   - Использование глаголов в URI не рекомендуется, так как операция определяется HTTP-методом:
     - Вместо `/getUser/123`, лучше использовать `/users/123/` с методом GET.
     - Вместо `/deleteOrder/456`, лучше использовать `/orders/456/` с методом DELETE.

2. **Принципы RESTful-адресации**

   Использование правильных URI позволяет соблюсти соглашения REST и делает API удобным для использования и поддержки. Примеры операций:
   
   ```plaintext
   GET /api/users/              # Получение списка пользователей
   POST /api/users/             # Создание нового пользователя
   GET /api/users/123/          # Получение пользователя с ID 123
   PUT /api/users/123/          # Полное обновление данных пользователя с ID 123
   PATCH /api/users/123/        # Частичное обновление данных пользователя с ID 123
   DELETE /api/users/123/       # Удаление пользователя с ID 123
   ```

#### **Вложенные маршруты**

**Вложенные маршруты** используются, когда ресурс имеет зависимые подресурсы. Это создает иерархическую структуру URI, что облегчает работу с связанными данными.

##### **Принципы вложенных маршрутов**

1. **Применение вложенных URI**
   - Вложенные URI логичны для работы с ресурсами, которые зависят от «родительского» ресурса, например:

     ```plaintext
     GET /api/orders/123/items/     # Получение списка позиций для заказа с ID 123
     POST /api/orders/123/items/    # Добавление новой позиции в заказ с ID 123
     GET /api/orders/123/items/456/ # Получение позиции с ID 456 в заказе с ID 123
     ```
     
   - Такой подход удобен для организации API и упрощает доступ к данным. В то же время слишком глубокая вложенность может ухудшить читаемость URI, рекомендуется ограничивать вложенность до двух уровней.


#### **<span style="color: #85004B;">Примеры на Django</span>**

На примере Django создадим REST API с моделями <span style="color: #85004B;">User</span>, <span style="color: #85004B;">Order</span> и <span style="color: #85004B;">OrderItem</span> и настроим маршруты с помощью Django Rest Framework.

##### **Структура проекта**

Проект Django с приложением `api` будет включать следующие файлы:

```plaintext
myproject/
├── myproject/
│   └── settings.py
└── api/
    ├── models.py         # Модели для пользователей и заказов
    ├── views.py          # Представления (views) для логики обработки
    ├── serializers.py    # Сериализаторы для преобразования моделей в JSON
    └── urls.py           # Настройка маршрутов для API
```

##### **Настройка URL в Django**

Маршруты для работы с основными и вложенными ресурсами описываются в `urls.py`. 

1. **Создание маршрутов для основных ресурсов**

   В файле `api/urls.py` добавим маршруты для работы с пользователями и заказами:

In [ ]:
# api/urls.py
from django.urls import path
from . import views

urlpatterns = [
    # Маршруты для пользователей
    path('users/', views.UserListCreateView.as_view(), name='user-list-create'),
    path('users/<int:pk>/', views.UserDetailView.as_view(), name='user-detail'),

    # Маршруты для заказов
    path('orders/', views.OrderListCreateView.as_view(), name='order-list-create'),
    path('orders/<int:pk>/', views.OrderDetailView.as_view(), name='order-detail'),
]

Подключаем `api/urls.py` к корневому `urls.py`:

In [ ]:
# myproject/urls.py
from django.urls import path, include

urlpatterns = [
    path('api/', include('api.urls')),  # Подключение API маршрутов
]

2. **Настройка вложенных маршрутов для позиций заказа**

   Добавим вложенные маршруты для позиций заказов в `api/urls.py`:

In [ ]:
# api/urls.py
from django.urls import path
from . import views

urlpatterns = [
    # Основные маршруты
    path('users/', views.UserListCreateView.as_view(), name='user-list-create'),
    path('users/<int:pk>/', views.UserDetailView.as_view(), name='user-detail'),
    path('orders/', views.OrderListCreateView.as_view(), name='order-list-create'),
    path('orders/<int:pk>/', views.OrderDetailView.as_view(), name='order-detail'),

    # Вложенные маршруты для позиций заказа
    path('orders/<int:order_id>/items/', views.OrderItemListView.as_view(), name='order-item-list'),
    path('orders/<int:order_id>/items/<int:item_id>/', views.OrderItemDetailView.as_view(), name='order-item-detail'),
]

##### **Создание представлений (views) для обработки запросов**

В `views.py` создаются классы для обработки запросов по каждому маршруту. Для этого используются классы `APIView` из Django REST Framework. 

In [ ]:
# api/views.py
from rest_framework.views import APIView
from rest_framework.response import Response
from rest_framework import status
from .models import User, Order, OrderItem
from .serializers import UserSerializer, OrderSerializer, OrderItemSerializer

# Представления для пользователей
class UserListCreateView(APIView):
    def get(self, request):
        users = User.objects.all()
        serializer = UserSerializer(users, many=True)
        return Response(serializer.data)

    def post(self, request):
        serializer = UserSerializer(data=request.data)
        if serializer.is_valid():
            serializer.save()
            return Response(serializer.data, status=status.HTTP_201_CREATED)
        return Response(serializer.errors, status=status.HTTP_400_BAD_REQUEST)

class UserDetailView(APIView):
    def get(self, request, pk):
        user = User.objects.get(pk=pk)
        serializer = UserSerializer(user)
        return Response(serializer.data)

    def put(self, request, pk):
        user = User.objects.get(pk=pk)
        serializer = UserSerializer(user, data=request.data)
        if serializer.is_valid():
            serializer.save()
            return Response(serializer.data)
        return Response(serializer.errors, status=status.HTTP_400_BAD_REQUEST)

    def delete(self, request, pk):
        user = User.objects.get(pk=pk)
        user.delete()
        return Response(status=status.HTTP_204_NO_CONTENT)

# Представления для вложенных ресурсов (позиции заказов)
class OrderItemListView(APIView):
    def get(self, request, order_id):
        items = OrderItem.objects.filter(order_id=order_id)
        serializer = OrderItemSerializer(items, many=True)
        return Response(serializer.data)

    def post(self, request, order_id):
        request.data['order'] = order_id  # Указываем order_id для создания связанного элемента
        serializer = OrderItemSerializer(data=request.data)
        if serializer.is_valid():
            serializer.save()
            return Response(serializer.data, status=status.HTTP_201_CREATED)
        return Response(serializer.errors, status=status.HTTP_400_BAD_REQUEST)

**<span style="color: #85004B; text-align: center;">Таким образом, у нас есть полноценный REST API с правильной структурой URI для основных и вложенных ресурсов, а также обработкой запросов с использованием Django REST Framework.</span>**